In [85]:
from utils import *
import random

/Users/jinanjiang/miniconda3/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [86]:
def write_to_file(contents, path):
    with open(path, 'w') as f:
        f.write(contents)
    
def read_from_file(path):
    with open(path) as f:
        return ''.join(f.readlines())
    
def find_sub_list(sl,l):
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind,ind+sll-1
    return -9999, -9999

In [260]:
def get_refactored_function_codes(cur_codes):
    splitted_codes = cur_codes.split('\n')

    signature_inds = [i for i in range(len(splitted_codes)) if splitted_codes[i].strip().startswith('function')]

    prefix = splitted_codes[:signature_inds[0]]
    postfix = '}'

    code_segments = []
    for i in range(len(signature_inds)):
        cur_ind = signature_inds[i]
        cur_paragraph = list(prefix)

        for j in range(len(signature_inds)):
            if j == i:
                if j < len(signature_inds) - 1:
                    cur_paragraph += splitted_codes[signature_inds[j]:signature_inds[j+1]]
                    reserved_length = len(splitted_codes[signature_inds[j]:signature_inds[j+1]])
                else:
                    cur_paragraph += splitted_codes[signature_inds[j]:-1]
                    reserved_length = len(splitted_codes[signature_inds[j]:-1])
            else:
                # just keep the signature
                cur_paragraph += [splitted_codes[signature_inds[j]]]

        cur_paragraph += postfix

        code_segments.append((list(cur_paragraph), get_function_name(splitted_codes[cur_ind]), splitted_codes[cur_ind], reserved_length))
    return code_segments

In [88]:
def make_a_folder_for_each_code_file():
    import os
    
    for f in [f for f in os.listdir('prompts_folder/function_dataset_builder/codes') if f.endswith('sol')]:
        if f.split('.')[0] not in os.listdir('prompts_folder/function_dataset_builder'):
            os.system('mkdir ' + 'prompts_folder/function_dataset_builder/' + f.split('.')[0])

In [239]:
# Notes for new codes: 
In the new generation, keep taking until seeing the first line that contains 'function' or "constructor"

In [154]:
codes_paths

['EIP20NonStandardInterface.sol',
 'ComptrollerInterface.sol',
 'Initializable.sol',
 'Roles.sol',
 'IERC20.sol',
 'SafeCast160.sol',
 'Comptroller.sol',
 'Exponential.sol',
 'PriceOracle.sol',
 'CToken.sol',
 'Dao.sol',
 'Pool.sol',
 'ERC1967UpgradeUpgradeable.sol',
 'ExponentialNoError.sol',
 'SafeERC20.sol',
 'SafeMath_2.sol',
 'SafeMath.sol',
 'ErrorReporter.sol',
 'Unitroller.sol',
 'IBeaconUpgradeable.sol',
 'Dispatcher.sol',
 'Address.sol',
 'governance.sol',
 'CarefulMath.sol',
 'EIP20Interface.sol',
 'InterestRateModel.sol',
 'ComptrollerStorage.sol',
 'IERC20Detailed.sol',
 'GovernorBravoInterfaces.sol',
 'INV.sol',
 'Comp.sol',
 'Context.sol',
 'SafeTransferLib.sol',
 'RewardsCollector.sol',
 'LockAndMsgSender.sol',
 'IERC165.sol',
 'AddressUpgradeable.sol',
 'UUPSUpgradeable.sol',
 'CTokenInterfaces.sol',
 'StorageSlotUpgradeable.sol',
 'Admin.sol']

In [192]:
codes_path = [f for f in os.listdir('prompts_folder/function_dataset_builder/codes') if f.endswith('sol')]

In [218]:
len(codes_path)

40

In [261]:
f_lst = []
f_count = 0

for p in codes_path[:1]:
    p_name = p.split('.')[0]
    print(p_name)

    
    cur_code_path = 'prompts_folder/function_dataset_builder/codes/' + p
    cur_codes = read_from_file(cur_code_path)
    cur_code_segments = get_refactored_function_codes(cur_codes)
    
    for seg in cur_code_segments:
        func_name = seg[1]
        reserved_length = seg[3]
        print(reserved_length)
        print(seg[0])
        
        func_codes = ''.join([i + '\n' for i in seg[0]])
        print(func_name)
        f_count += 1
        f_lst.append(func_name)
        
        GPT_prefix = '\n'
        GPT_prefix += 'The function that I want you to summarize is ' + func_name + ', which is defined in the '
        GPT_prefix += 'codes given below. Please note that I have hidden the definition of all other functions, '
        GPT_prefix += 'but I still kept their signature, with the purpose of providing richer background information'
        GPT_prefix += ' for you. The codes are: \n'
        
        func_codes = GPT_prefix + func_codes
        write_to_file(func_codes, 'prompts_folder/function_dataset_builder/' + p_name + '/' + func_name + '.txt')

EIP20NonStandardInterface
7
['pragma solidity ^0.5.16;', '', '/**', ' * @title EIP20NonStandardInterface', ' * @dev Version of ERC20 with no return values for `transfer` and `transferFrom`', ' *  See https://medium.com/coinmonks/missing-return-value-bug-at-least-130-tokens-affected-d67bf08521ca', ' */', 'interface EIP20NonStandardInterface {', '', '    /**', '     * @notice Get the total number of tokens in circulation', '     * @return The supply of tokens', '     */', '    function totalSupply() external view returns (uint256);', '', '    /**', '     * @notice Gets the balance of the specified address', '     * @param owner The address from which the balance will be retrieved', '     * @return The balance', '     */', '    function balanceOf(address owner) external view returns (uint256 balance);', '    function transfer(address dst, uint256 amount) external;', '    function transferFrom(address src, address dst, uint256 amount) external;', '    function approve(address spender, uint

In [263]:
print(''.join([i+'\n' for i in ['pragma solidity ^0.5.16;', '', '/**', ' * @title EIP20NonStandardInterface', ' * @dev Version of ERC20 with no return values for `transfer` and `transferFrom`', ' *  See https://medium.com/coinmonks/missing-return-value-bug-at-least-130-tokens-affected-d67bf08521ca', ' */', 'interface EIP20NonStandardInterface {', '', '    /**', '     * @notice Get the total number of tokens in circulation', '     * @return The supply of tokens', '     */', '    function totalSupply() external view returns (uint256);', '    function balanceOf(address owner) external view returns (uint256 balance);', '    function transfer(address dst, uint256 amount) external;', '    function transferFrom(address src, address dst, uint256 amount) external;', '', '    /**', '      * @notice Approve `spender` to transfer up to `amount` from `src`', '      * @dev This will overwrite the approval amount for `spender`', '      *  and is subject to issues noted [here](https://eips.ethereum.org/EIPS/eip-20#approve)', '      * @param spender The address of the account which may transfer tokens', '      * @param amount The number of tokens that are approved', '      * @return Whether or not the approval succeeded', '      */', '    function approve(address spender, uint256 amount) external returns (bool success);', '    function allowance(address owner, address spender) external view returns (uint256 remaining);', '}']]))

pragma solidity ^0.5.16;

/**
 * @title EIP20NonStandardInterface
 * @dev Version of ERC20 with no return values for `transfer` and `transferFrom`
 *  See https://medium.com/coinmonks/missing-return-value-bug-at-least-130-tokens-affected-d67bf08521ca
 */
interface EIP20NonStandardInterface {

    /**
     * @notice Get the total number of tokens in circulation
     * @return The supply of tokens
     */
    function totalSupply() external view returns (uint256);
    function balanceOf(address owner) external view returns (uint256 balance);
    function transfer(address dst, uint256 amount) external;
    function transferFrom(address src, address dst, uint256 amount) external;

    /**
      * @notice Approve `spender` to transfer up to `amount` from `src`
      * @dev This will overwrite the approval amount for `spender`
      *  and is subject to issues noted [here](https://eips.ethereum.org/EIPS/eip-20#approve)
      * @param spender The address of the account which may transfer toke

In [211]:
f_count

626

In [216]:
len(set(f_lst))

291

In [248]:
# to count the number of output files

cur_code_path = 'prompts_folder/function_dataset_builder/'
l = []
for i in [i for i in os.listdir(cur_code_path) if not i.endswith('.txt') and i != 'codes' and not i.startswith('.')]:
    l.extend(os.listdir(cur_code_path + i))


In [249]:
len([i for i in l if not i.endswith('_cache.txt')])


518

In [250]:
len([i for i in l if i.endswith('_cache.txt') and not i.endswith('filled_in_codes_cache.txt')])


151

In [233]:
openai.api_key = "8b116907e4144bbf90b7e15d56e54938"

In [234]:
class ChatGPT:
    
    def __init__(self):
        self.clear_context()
    
    def clear_context(self):
        self.context_messages = None
        self.num_of_contexts = 0
        
    def get_response(self, prompt, is_trivial = False):
        def get_response_inner(prompt, context_messages = None):
            d = {"role": "user", "content": prompt}
            if context_messages is None:
                new_context_messages = [d]
            else:
                new_context_messages = context_messages + [d]
            completion = openai.ChatCompletion.create(engine="chatgpt-4", messages=new_context_messages)
            gpt_response = completion.choices[0].message.content

            response_d = {"role": "system", "content": gpt_response}
            new_context_messages = new_context_messages + [response_d]

            return gpt_response, new_context_messages
        
        gpt_response, context_messages = get_response_inner(prompt, context_messages = self.context_messages)
        
        print("num_of_contexts: ", self.num_of_contexts)
        
        if not is_trivial:
            self.context_messages = context_messages
            self.num_of_contexts += 1
        
        return gpt_response
    
    def saves_dialogues(self, filename):
        with open('saved_dialogues/' + filename + '.json', 'w') as f:
            json.dump(self.context_messages, f)
        

In [92]:
def summarize_functions(input_codes, use_cache = False, save_dialogue = False, cache_prefix = 'cache/', collection_mode = False):
    cache_path = cache_prefix
    
    if use_cache and not collection_mode:
        if os.path.exists(cache_path):
            return read_from_file(cache_path), True
        
    source_files_lst = []
        
    l = []
    # source_files_lst.append(['prompts_folder/summarize_contracts.txt', "ERC20.sol"])
    l.extend(['prompts_folder/function_dataset_builder/' + 'summarize_functions_dataset_builder' + '.txt', GPT_string(input_codes)])
    
    source_files_lst.append(l)

    if save_dialogue:
        gpt, all_responses_lst = do_pipeline(source_files_lst, output_file_path = get_filename_from_path(contract_file_name) + '_relevant_contracts', do_print = True)
    else:
        gpt, all_responses_lst = do_pipeline(source_files_lst, output_file_path = None, do_print = True)

    assert len(all_responses_lst) == 1
    
    if use_cache:
        # assert not os.path.exists(cache_path)
        content = all_responses_lst[0]
        write_to_file(content, cache_path)
        
    return all_responses_lst[0], False



In [237]:
summarized_functions = []

for p in codes_path:
    p_name = p.split('.')[0]
    
    cur_code_path = 'prompts_folder/function_dataset_builder/codes/' + p
    cur_codes = read_from_file(cur_code_path)
    cur_code_segments = get_refactored_function_codes(cur_codes)
    for seg in cur_code_segments:
        func_name = seg[1]
        func_codes = ''.join([i + '\n' for i in seg[0]])
        
        if func_name in summarized_functions:
            continue
        
        GPT_prefix = '\n'
        GPT_prefix += 'The function that I want you to summarize is ' + func_name + ', which is defined in the '
        GPT_prefix += 'codes given below. Please note that I have hidden the definition of all other functions, '
        GPT_prefix += 'but I still kept their signature, with the purpose of providing richer background information'
        GPT_prefix += ' for you. The codes are: \n'
        
        func_codes = GPT_prefix + func_codes
        
        _, cache_used = summarize_functions(func_codes, use_cache = True, cache_prefix = 'prompts_folder/function_dataset_builder/' + p_name + '/' + func_name + '_cache.txt', collection_mode = False)

        summarized_functions.append(func_name)
        
        print(func_name, p_name)
        if not cache_used:
            time.sleep(random.randint(20, 35))
        else:
            print('cache_used')
        

totalSupply EIP20NonStandardInterface
cache_used
balanceOf EIP20NonStandardInterface
cache_used
transfer EIP20NonStandardInterface
cache_used
transferFrom EIP20NonStandardInterface
cache_used
approve EIP20NonStandardInterface
cache_used
allowance EIP20NonStandardInterface
cache_used
enterMarkets ComptrollerInterface
cache_used
exitMarket ComptrollerInterface
cache_used
mintAllowed ComptrollerInterface
cache_used
mintVerify ComptrollerInterface
cache_used
redeemAllowed ComptrollerInterface
cache_used
redeemVerify ComptrollerInterface
cache_used
borrowAllowed ComptrollerInterface
cache_used
borrowVerify ComptrollerInterface
cache_used
repayBorrowAllowed ComptrollerInterface
cache_used
repayBorrowVerify ComptrollerInterface
cache_used
liquidateBorrowAllowed ComptrollerInterface
cache_used
liquidateBorrowVerify ComptrollerInterface
cache_used
seizeAllowed ComptrollerInterface
cache_used
seizeVerify ComptrollerInterface
cache_used
transferAllowed ComptrollerInterface
cache_used
transferVeri

APIError: Invalid response object from API: '{ "statusCode": 403, "message": "Out of call volume quota. Quota will be replenished in 15:57:27." }' (HTTP response code was 403)

In [131]:
def fill_in_missing_functions(append_texts, use_cache = False, save_dialogue = False, cache_prefix = 'cache/', collection_mode = False):
    cache_path = cache_prefix
    
    if use_cache and not collection_mode:
        if os.path.exists(cache_path):
            return read_from_file(cache_path), True
        
    source_files_lst = []
        
    l = []
    # source_files_lst.append(['prompts_folder/summarize_contracts.txt', "ERC20.sol"])
    l.extend(['prompts_folder/function_dataset_builder/' + 'generate_missing_codes' + '.txt', GPT_string(append_texts)])
    
    source_files_lst.append(l)

    if save_dialogue:
        gpt, all_responses_lst = do_pipeline(source_files_lst, output_file_path = get_filename_from_path(contract_file_name) + '_relevant_contracts', do_print = True)
    else:
        gpt, all_responses_lst = do_pipeline(source_files_lst, output_file_path = None, do_print = True)

    assert len(all_responses_lst) == 1
    
    if use_cache:
        # assert not os.path.exists(cache_path)
        content = all_responses_lst[0]
        write_to_file(content, cache_path)
        
    return all_responses_lst[0], False



In [238]:
codes_path

['EIP20NonStandardInterface.sol',
 'ComptrollerInterface.sol',
 'Initializable.sol',
 'Roles.sol',
 'IERC20.sol',
 'SafeCast160.sol',
 'Comptroller.sol',
 'Exponential.sol',
 'PriceOracle.sol',
 'CToken.sol',
 'Dao.sol',
 'Pool.sol',
 'ERC1967UpgradeUpgradeable.sol',
 'ExponentialNoError.sol',
 'SafeERC20.sol',
 'SafeMath_2.sol',
 'SafeMath.sol',
 'ErrorReporter.sol',
 'Unitroller.sol',
 'IBeaconUpgradeable.sol',
 'Dispatcher.sol',
 'Address.sol',
 'governance.sol',
 'CarefulMath.sol',
 'EIP20Interface.sol',
 'InterestRateModel.sol',
 'IERC20Detailed.sol',
 'GovernorBravoInterfaces.sol',
 'INV.sol',
 'Comp.sol',
 'Context.sol',
 'SafeTransferLib.sol',
 'RewardsCollector.sol',
 'LockAndMsgSender.sol',
 'IERC165.sol',
 'AddressUpgradeable.sol',
 'UUPSUpgradeable.sol',
 'CTokenInterfaces.sol',
 'StorageSlotUpgradeable.sol',
 'Admin.sol']

In [252]:
[1,2,3].index(2)

1

In [253]:
def remove_given_func_name(cur_code_path, cur_func_name):
    assert not cur_func_name.endswith('_cache')
    
    code_file = read_from_file(cur_code_path + '/' + cur_func_name + '.txt')
    splitted_codes = code_file.split('The codes are: \n')[1]
    splitted_codes = splitted_codes.split('\n')

    signature_inds = [i for i in range(len(splitted_codes)) if splitted_codes[i].strip().startswith('function')]
    target_func_index = [i for i in range(len(splitted_codes)) if splitted_codes[i].strip().startswith('function ' + cur_func_name + '(')]
    
    if len(target_func_index) == 1:
        target_func_index = target_func_index[0]
    else:
        filtered_target_ind = []
        for ind in target_func_index:
            t222 = signature_inds.index(ind)
            if t222 != len(signature_inds) - 1 and signature_inds[t222 + 1] > ind + 1:
                filtered_target_ind.append(t222)
        assert len(filtered_target_ind) == 1
        target_func_index = filtered_target_ind[0]
    
    
    assert target_func_index in signature_inds

    i = signature_inds.index(target_func_index)

    beginning_ind = signature_inds[i]
    ending_ind = None
    if i < len(signature_inds) - 1:
        ending_ind = signature_inds[i + 1]
    else:
        assert i == len(signature_inds) - 1
        ending_ind = -1


    splitted_codes = splitted_codes[:beginning_ind] + splitted_codes[ending_ind:-1]
    return splitted_codes

In [254]:
for p in codes_path:
    p_name = p.split('.')[0]
    
    cur_code_path = 'prompts_folder/function_dataset_builder/' + p_name
    all_files = os.listdir(cur_code_path)
    files_with_summary = [i for i in all_files if i.endswith('_cache.txt') and not i.endswith('filled_in_codes_cache.txt')]

    
    for f in files_with_summary:
            print(f, p_name)
            # e.g. 'borrowFresh_cache.txt'
            append_texts = ''
            cur_func_name = f.split('_cache')[0]

            function_summary = read_from_file(cur_code_path + '/' + f)
            new_codes_with_fun_removed = ''.join([i+'\n' for i in remove_given_func_name(cur_code_path, cur_func_name)])

            append_texts += 'Here are the codes of the contract for which you will write a new function:\n'
            append_texts += new_codes_with_fun_removed + '\n\n'

            append_texts += 'The summary of the function that I want you to add is given below:\n'
            append_texts += function_summary

            _, cache_used = fill_in_missing_functions(append_texts, use_cache = True, cache_prefix = 'prompts_folder/function_dataset_builder/' + p_name + '/' + cur_func_name + 'filled_in_codes_cache.txt', collection_mode = False)

            
            if not cache_used:
                time.sleep(random.randint(20, 35))
            else:
                print('cache_used')
            print('DONE: ', f, p_name)


totalSupply_cache.txt EIP20NonStandardInterface
cache_used
DONE:  totalSupply_cache.txt EIP20NonStandardInterface
allowance_cache.txt EIP20NonStandardInterface
cache_used
DONE:  allowance_cache.txt EIP20NonStandardInterface
balanceOf_cache.txt EIP20NonStandardInterface
cache_used
DONE:  balanceOf_cache.txt EIP20NonStandardInterface
transferFrom_cache.txt EIP20NonStandardInterface
cache_used
DONE:  transferFrom_cache.txt EIP20NonStandardInterface
approve_cache.txt EIP20NonStandardInterface
cache_used
DONE:  approve_cache.txt EIP20NonStandardInterface
transfer_cache.txt EIP20NonStandardInterface
cache_used
DONE:  transfer_cache.txt EIP20NonStandardInterface
mintAllowed_cache.txt ComptrollerInterface
cache_used
DONE:  mintAllowed_cache.txt ComptrollerInterface
enterMarkets_cache.txt ComptrollerInterface
cache_used
DONE:  enterMarkets_cache.txt ComptrollerInterface
exitMarket_cache.txt ComptrollerInterface
cache_used
DONE:  exitMarket_cache.txt ComptrollerInterface
redeemAllowed_cache.txt 

KeyboardInterrupt: 

In [ ]:
# feedback optimization
target_contract_name = 'Comptroller_0xBafE01ff935C7305907c33BF824352eE5979B526'
comptroller_coldes_path = get_all_code_files_path(target_contract_name)

task_name = 'task2'
generated_codes = summarize_all_codes(comptroller_coldes_path, task_name)

# beginning a multi-round improvement
MAX_ROUNDS = 5
cur_round = 0
while cur_round < MAX_ROUNDS:
    code_quality_feedback = get_code_quality_feedback(generated_codes, task_name, cur_round = cur_round, use_cache = True, cache_prefix = 'cache/' + task_name + '/')
    fixed_codes = fix_codes_based_on_feedback(generated_codes, task_name, cur_round = cur_round, use_cache = True, cache_prefix = 'cache/' + task_name + '/')

    need_improvement = check_stopping_criteria(fixed_codes, task_name, cur_round, use_cache = True, cache_prefix = 'cache/' + task_name + '/')
    is_true = int('True' in need_improvement)
    is_false = int('False' in need_improvement)

    assert is_true ^ is_false
    assert is_true != is_false

    cur_round += 1
    
    if is_false:
        break
    else:
        continue


In [ ]:
# gas optimization

def check_gas_pattern(target_codes, pattern_name, use_cache = False, save_dialogue = False, cache_prefix = 'cache/'):
    cache_path = cache_prefix + pattern_name + '.txt'
    
    if use_cache:
        if os.path.exists(cache_path):
            return read_from_file(cache_path)
        
    source_files_lst = []
    
    l = []
    # source_files_lst.append(['prompts_folder/summarize_contracts.txt', "ERC20.sol"])
    l.extend(['prompts_folder/' + pattern_name + '.txt', GPT_string(target_codes)])
    
    source_files_lst.append(l)

    if save_dialogue:
        gpt, all_responses_lst = do_pipeline(source_files_lst, output_file_path = get_filename_from_path(contract_file_name) + '_relevant_contracts', do_print = True)
    else:
        gpt, all_responses_lst = do_pipeline(source_files_lst, output_file_path = None, do_print = True)

    assert len(all_responses_lst) == 1
    
    if use_cache:
        assert not os.path.exists(cache_path)
        content = all_responses_lst[0]
        write_to_file(content, cache_path)
        
    return all_responses_lst[0]

target_codes = read_from_file('dao_code_repository/0xbafe01ff935c7305907c33bf824352ee5979b526/contracts/CarefulMath.sol')
target_codes = remove_tags(target_codes)

pattern_names = ['1_dead_code', '2_Opaque_predicate', '3_Expensive_operations_in_a_loop', 
                '4_Constant_outcome_of_a_loop', '5_Loop_fusion',
                '6_Repeated_computations_in_a loop', '7_Comparison with_unilateral_outcome_in_a_loop']

pattern_names = ['gas_reduction/2_Opaque_predicate']
#pattern_names = ['gas_reduction/simple_gas_finder']
for pattern_name in pattern_names:
    issues_found = check_gas_pattern(target_codes, pattern_name, use_cache = True)
    print('DONE: ', pattern_name)



'pragma solidity ^0.5.16;\n\nimport "./ComptrollerInterface.sol";\nimport "./CTokenInterfaces.sol";\nimport "./ErrorReporter.sol";\nimport "./Exponential.sol";\nimport "./EIP20Interface.sol";\nimport "./InterestRateModel.sol";\n\n/**\n * @title Compound\'s CToken Contract\n * @notice Abstract base for CTokens\n * @author Compound\n */\ncontract CToken is CTokenInterface, Exponential, TokenErrorReporter {\n    /**\n     * @notice Initialize the money market\n     * @param comptroller_ The address of the Comptroller\n     * @param interestRateModel_ The address of the interest rate model\n     * @param initialExchangeRateMantissa_ The initial exchange rate, scaled by 1e18\n     * @param name_ EIP-20 name of this token\n     * @param symbol_ EIP-20 symbol of this token\n     * @param decimals_ EIP-20 decimal precision of this token\n     */\n    function initialize(ComptrollerInterface comptroller_,\n    function transferTokens(address spender, address src, address dst, uint tokens) inter

In [ ]:
def get_refactored_function_codes_excluding_function(cur_codes):
    splitted_codes = cur_codes.split('\n')

    signature_inds = [i for i in range(len(splitted_codes)) if splitted_codes[i].strip().startswith('function')]

    prefix = splitted_codes[:signature_inds[0]]
    postfix = '}'

    code_segments = []
    for i in range(len(signature_inds)):
        cur_ind = signature_inds[i]
        cur_paragraph = list(prefix)

        for j in range(len(signature_inds)):
            if j == i:
                if j < len(signature_inds) - 1:
                    cur_paragraph += splitted_codes[signature_inds[j]:signature_inds[j+1]]
                else:
                    cur_paragraph += splitted_codes[signature_inds[j]:-1]
            else:
                # just keep the signature
                cur_paragraph += [splitted_codes[signature_inds[j]]]

        cur_paragraph += postfix

        code_segments.append((list(cur_paragraph), get_function_name(splitted_codes[cur_ind]), splitted_codes[cur_ind]))
    return code_segments

In [ ]:
cur_codes = read_from_file('prompts_folder/function_dataset_builder/codes/CarefulMath.sol')

In [62]:
code_segments = get_refactored_function_codes(cur_codes)

In [63]:
code_segments[2]

(['pragma solidity ^0.5.16;',
  '',
  '/**',
  '  * @title Careful Math',
  '  * @author Compound',
  "  * @notice Derived from OpenZeppelin's SafeMath library",
  '  *         https://github.com/OpenZeppelin/openzeppelin-solidity/blob/master/contracts/math/SafeMath.sol',
  '  */',
  'contract CarefulMath {',
  '',
  '    /**',
  '     * @dev Possible error codes that we can return',
  '     */',
  '    enum MathError {',
  '        NO_ERROR,',
  '        DIVISION_BY_ZERO,',
  '        INTEGER_OVERFLOW,',
  '        INTEGER_UNDERFLOW',
  '    }',
  '',
  '    ',
  '    /**',
  '    * @dev Multiplies two numbers, returns an error on overflow.',
  '    */',
  '    function mulUInt(uint a, uint b) internal pure returns (MathError, uint) {',
  '    function divUInt(uint a, uint b) internal pure returns (MathError, uint) {',
  '    function subUInt(uint a, uint b) internal pure returns (MathError, uint) {',
  '        if (b <= a) {',
  '            return (MathError.NO_ERROR, a - b);',
  ' 

In [47]:
def get_function_name(signature):
    return signature.strip().split('(')[0].split('function ')[1]

['', 'mulUInt']

In [13]:
 cur_codes.split('\n')

['pragma solidity ^0.5.16;',
 '',
 '/**',
 '  * @title Careful Math',
 '  * @author Compound',
 "  * @notice Derived from OpenZeppelin's SafeMath library",
 '  *         https://github.com/OpenZeppelin/openzeppelin-solidity/blob/master/contracts/math/SafeMath.sol',
 '  */',
 'contract CarefulMath {',
 '',
 '    /**',
 '     * @dev Possible error codes that we can return',
 '     */',
 '    enum MathError {',
 '        NO_ERROR,',
 '        DIVISION_BY_ZERO,',
 '        INTEGER_OVERFLOW,',
 '        INTEGER_UNDERFLOW',
 '    }',
 '',
 '    ',
 '    /**',
 '    * @dev Multiplies two numbers, returns an error on overflow.',
 '    */',
 '    function mulUInt(uint a, uint b) internal pure returns (MathError, uint) {',
 '        if (a == 0) {',
 '            return (MathError.NO_ERROR, 0);',
 '        }',
 '',
 '        uint c = a * b;',
 '',
 '        if (c / a != b) {',
 '            return (MathError.INTEGER_OVERFLOW, 0);',
 '        } else {',
 '            return (MathError.NO_ERROR, c)